# Feature analysis for asthma-specific patients against control group

In [3]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, log_loss

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [4]:
# read dataset
ds = pd.read_pickle("../data/community_expr.pkl")

control = ds['control']
asthma = ds['asthma']

communities = list(asthma['community'].unique())

## Analysis of genes as predictors

In [31]:
## train logistic regression

# combine asthma and control patients into sets of input and labels
a2 = pd.DataFrame(asthma.transpose())
c2 = pd.DataFrame(control.transpose())
a2['asthma'] = 1
c2['asthma'] = 0
train_a = a2.head(len(a2) - 1)
train_c = c2.head(len(c2) - 1)
dataset = pd.concat([train_a, train_c])
X = dataset.drop('asthma', axis=1)
y = dataset['asthma']

## fit model
reg = LogisticRegression(max_iter=1000).fit(np.array(X), np.array(y))

# calculate score
loss = log_loss(y, reg.predict_proba(X))
print("Logistic loss: %.4f" % loss)

Logistic loss: 0.0949


In [32]:
# rank genes by coefficients
coefs = reg.coef_[0]

# sort by coef
maxn = sorted(coefs,reverse=True)[:20]
max_genes = {}
for n in maxn:
    i, = np.where(np.isclose(coefs, n))
    max_genes[X.columns[i][0]] = n

# output results
max_genes = pd.DataFrame(max_genes.items(), columns=['gene', 'coefficient'])
max_genes.to_csv('../data/gene_coefficients.csv')
max_genes.head()

,gene,coefficient
0,ST6GALNAC3,0.534319
1,TSPYL5,0.523160
2,PLA2G7,0.471735
3,MICU3,0.463369
4,DHRS4-AS1,0.455403


## Community-based analysis

In [30]:
# group genes into communities
a2 = pd.DataFrame(asthma.groupby('community').mean().transpose())
c2 = pd.DataFrame(control.groupby('community').mean().transpose())
a2['asthma'] = 1
c2['asthma'] = 0
train_a = a2.head(len(a2) - 1)
train_c = c2.head(len(c2) - 1)
dataset = pd.concat([train_a, train_c])
X = dataset.drop('asthma', axis=1)
y = dataset['asthma']

# rank communities
reg = LogisticRegression(max_iter=1000).fit(np.array(X), np.array(y))
coefs = reg.coef_[0]
maxn = sorted(coefs,reverse=True)
max_comms = {}
for n in maxn:
    i, = np.where(np.isclose(coefs, n))
    max_comms[X.columns[i][0]] = n

# output results
max_comms = pd.DataFrame(max_comms.items(), columns=['community', 'coefficient'])
max_comms.to_csv('../data/community_coefficients.csv')
max_comms.head()

,community,coefficient
0,Community 0,0.263610
1,Community 23,-0.050312
2,Community 48,-0.186279
3,Community 27,-0.203218
4,Community 2,-0.272948
